In [ ]:
# double check this, I have moved things

In [ ]:
import pandas,os,glob,numpy as np
vectors = sorted(glob.glob('multiLanguageFromDumpsSecLinks/*'), key=os.path.getsize)


In [ ]:
import json
def jaccard(x):
    a = set(json.loads(x[0].replace("'",'"')))
    b = set(json.loads(x[1].replace("'",'"')))
    return len(a.intersection(b)) / len(a.union(b)) , len(a.intersection(b)) 

In [ ]:
while vectors:
    lang1 = vectors.pop()
    print(lang1)
    lang1_code = lang1.split('_')[-1][:-4]
    df = pandas.read_csv(lang1,sep='\t',header=None).rename(columns={0:'q',1:lang1_code,2:'links'})
    df.set_index('q',inplace=True)
    #df = df[df.index.isin(articleFilter)]
    for lang2 in vectors:
        lang2_code = lang2.split('_')[-1][:-4]
        print(lang2)
        df2 = pandas.read_csv(lang2,sep='\t',header=None).rename(columns={0:'q',1:lang2_code,2:'links'})
        df2.set_index('q',inplace=True)
        #df2 = df2[df2.index.isin(articleFilter)]
        dfs = df.join(df2,how='inner',rsuffix='_2')
        pairs = zip(dfs['links'],dfs['links_2'])
        result = map(jaccard,pairs)
        #prepare output
        del(dfs['links'])
        del(dfs['links_2'])
        dfs.reset_index(inplace=True)
        dfs['jaccard'] = pandas.Series(result)
        dfs[['links_jaccard','links_common']] = pandas.DataFrame(dfs['jaccard'].values.tolist())
        del(dfs['jaccard'])
        dfs = dfs[[lang1_code,lang2_code,'links_jaccard','links_common']].groupby([lang1_code,lang2_code]).agg(['mean','median','sum']).reset_index()
        dfs.columns =['%s%s' % (x,y.upper()) for x,y in dfs.columns.tolist()]   #flatten index                                      
        dfs.to_pickle('linksSimilarities/%s_%s.p' % (lang1_code,lang2_code))
   